<a href="https://colab.research.google.com/github/gtbnhyujmj/Auto_Shorts_Maker/blob/PNG_MakeByYou/PNG_Size_MakeByYou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!apt-get -y install ffmpeg
!pip install --quiet pillow opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# 接入雲端硬碟

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# (def) 創造出透明背景的PNG檔，上面要壓印文字。

In [24]:
# 可以設定 font_size=128
# 整個PNG檔大小 image_size=(1080, 1920)
# 文字顏色 -> draw.text((x, y), text, font=font, fill=(0, 0, 0, 255))  # 黑字

def create_text_image(text, font_size=128, image_size=(1080, 1920), align='center', output_path="/content/text_overlay.png"):
    # 字體載入（NotoSansTC-VariableFont_wght.ttf 可以輸出中文）
    font_path = "/content/drive/MyDrive/shorts/font/NotoSansTC-VariableFont_wght.ttf"
    font = ImageFont.truetype(font_path, font_size)

    img = Image.new('RGBA', image_size, (0, 0, 0, 0))  # 透明背景
    draw = ImageDraw.Draw(img)

    # 計算文字寬高
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # 位置：置左 / 置中 / 置右
    if align == 'left':
        x = 20
    elif align == 'center':
        x = (image_size[0] - text_width) // 2
    else:  # right
        x = image_size[0] - text_width - 20

    y = (image_size[1] - text_height) // 2
    draw.text((x, y), text, font=font, fill=(0, 0, 0, 255))  # 黑字

    img.save(output_path)
    print(f"✅ PNG 文字圖片已儲存至 {output_path}")

# 把透明背景的PNG讀出來，黏貼到MP4檔上面

In [25]:
import cv2
import numpy as np

# 黏貼參考點 = position=(0, 0) = 右上角

def overlay_png_on_video(video_path, png_path, output_path, position=(0, 0)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    # 讀取 PNG 圖片 (RGBA)
    overlay = cv2.imread(png_path, cv2.IMREAD_UNCHANGED)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        overlay_resized = cv2.resize(overlay, (overlay.shape[1], overlay.shape[0]))
        x, y = position
        h_, w_, _ = overlay_resized.shape

        if x + w_ > w or y + h_ > h:
            print("⚠️ 文字超出畫面邊界")
            break

        # 將 overlay RGBA 拆成 BGR + Alpha
        b, g, r, a = cv2.split(overlay_resized)
        overlay_rgb = cv2.merge((b, g, r))
        mask = cv2.merge((a, a, a)) / 255.0

        roi = frame[y:y+h_, x:x+w_]
        blended = (1 - mask) * roi + mask * overlay_rgb
        frame[y:y+h_, x:x+w_] = blended.astype(np.uint8)

        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ 輸出完成: {output_path}")

# 執行區

In [26]:
# 文字生成 PNG
create_text_image("我愛你", align='center')

✅ PNG 文字圖片已儲存至 /content/text_overlay.png


In [27]:
# 貼到影片上
# 黏貼參考點 = position=(0, 0) = 右上角

video_path = "/content/drive/MyDrive/shorts/movie01/output.mp4"
output_video = "/content/output_final.mp4"

overlay_png_on_video(video_path, "/content/text_overlay.png", output_video, position=(0, 0))

✅ 輸出完成: /content/output_final.mp4


# 檢討

In [ ]:
# 在素材準備階段就直接，透明背景, 1080x1920, 整個程式會好寫個十倍。
# 直接當成貼透明貼紙 = 圖層，整個影片會更簡單製作。
# 這個做法直接丟去Git的分支，不處理了。

In [ ]:
# STU